In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc; reload(fcc)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline
def savefig(name):
    plt.savefig(name + ".png", dpi=300, bbox_inches='tight')
    plt.savefig(name + ".pdf", bbox_inches='tight')


In [ ]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

# Data loading

In [ ]:
carbons = np.arange(1,8)

In [ ]:
reload(pu_delayfb)
carbons = [1,2,3,4,5,6,7]

single_C_decay_data = dict()

for c in carbons:
    data = dict()
    data["x"], data["y"], data["y_u"], data["fr"] = \
    pu_delayfb.number_of_repetitions_stitched(
        contains = 'sweep_number_of_reps_C%d_X' % (c),
        older_thans = ["20170912000000"],
        do_fit = True,
        fixed=[0,2,5,6,4],
        T2star_correction=True,
        LDE_element_length=7e-6,
        ret_data_fit=True)
    single_C_decay_data[c] = data

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)
import itertools

carbon_combis = list(itertools.combinations(carbons, 2))

freqs = np.zeros((len(carbons) + 2*len(carbon_combis), 3)) # -1, 0, +1
# T2stars = np.zeros((len(carbons) + 2*len(carbon_combis)))

c_idxs = dict()

for i_c, c in enumerate(carbons):
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    msp1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, "p1")]
    msm1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, "m1")]
#     T2star = msmt_params.cfg['samples']['111no2']['C%d_T2star_0' % (c, "m1")]
    
    freqs[i_c,0] = msm1_freq
    freqs[i_c,1] = ms0_freq
    freqs[i_c,2] = msp1_freq
    
    print("C%d dephasing rate parameter: %.3f kHz" % (c, 1e-3 * (msp1_freq-msm1_freq)) )
#     T2stars[i_c] = T2star
    
    c_idxs[str(c)] = i_c
    
# 
# +1 frequencies of C4 and C5 are interchanged
freqs[3:5,2] = freqs[3:5,2][::-1]
    
for i_cc, cc in enumerate(carbon_combis):
    idx = 2*i_cc + len(carbons)
    freqs[idx,:] = (freqs[c_idxs[str(cc[0])],:] + freqs[c_idxs[str(cc[1])],:])
    print("C%s dephasing rate parameter: %.3f kHz" % ("%d%d+" % cc, 1e-3 * (freqs[idx,2] - freqs[idx,0])) )
    c_idxs["%d%d+" % cc] = idx
    freqs[idx+1,:] = (freqs[c_idxs[str(cc[0])],:] - freqs[c_idxs[str(cc[1])],:])
    print("C%s dephasing rate parameter: %.3f kHz" % ("%d%d-" % cc, 1e-3 * (freqs[idx+1,2] - freqs[idx+1,0])) )
    c_idxs["%d%d-" % cc] = idx+1
    
print freqs

In [ ]:
testcombi = "24+"
testidx = c_idxs[testcombi]
testidx_1 = c_idxs[testcombi[0]]
testidx_2 = c_idxs[testcombi[1]]

print freqs[testidx]
print np.diff(freqs[testidx])
print freqs[testidx_1]
print np.diff(freqs[testidx_1])
print freqs[testidx_2]
print np.diff(freqs[testidx_2])


# Markov chain model

## State space

state | index
--- | ---
0 | 0
-1 | 1
+1 | 2
E' | 3
S | 4

In [ ]:
# rates

excitation_rate = 1 / 70e-9

Ep_branching = np.array([0.03, 0.94, 0.03, 0.0, 0.63])
Ep_lifetime = 11.7e-9

S_branching = np.array([9., 1., 1., 0., 0.])
S_lifetime = 300e-9

# normalization
Ep_branching = Ep_branching / np.sum(Ep_branching)
S_branching = S_branching / np.sum(S_branching)

Ep_rate = 1 / Ep_lifetime
S_rate = 1 / S_lifetime

In [ ]:
# construct generator matrix
num_states = 5

G = np.zeros((num_states, num_states))

# rates leaving state 0
G[0,:] = 0.0
# rates leaving state -1
G[1,3] = excitation_rate
# rates leaving state +1
G[2,3] = excitation_rate
# rates leaving state E'
G[3,:] = Ep_rate * Ep_branching
# rates leaving state S
G[4,:] = S_rate * S_branching

# compute diagonal elements
diagonal_sum = np.sum(np.abs(G*np.eye(num_states)))
if diagonal_sum > 0:
    print("WARNING! There are diagonal rates in the generator matrix")
    
G[np.diag_indices_from(G)] = -1. * np.sum(G, axis=1)

In [ ]:
print("Generator matrix in MHz")
print(str(G / 1e6))

In [ ]:
stationary_dist = np.array([1., 0., 0., 0., 0.])
np.dot(stationary_dist, G)

In [ ]:
import numpy.linalg as la

# la.solve(G, np.zeros((num_states)))

# Binomial - geometric model

In [ ]:
from scipy import stats
from numpy import random

singlet_lifetime = 300e-9
repump_success = 9./11.
excitation_time = 40e-9

t_repump = np.linspace(0,10e-6,1000)

In [ ]:
plt.plot(t_repump*1e6, stats.erlang.pdf(t_repump/singlet_lifetime, 3))

In [ ]:
def compound_geom_erlang_dist(x, r, tau):
    sum_cutoff = 100
    pdf = np.zeros_like(x, dtype=np.double)
    for i in xrange(1,sum_cutoff):
        pdf += stats.geom.pmf(i, r) * stats.erlang.pdf(x/tau, i)
        
    return pdf

In [ ]:
plt.plot(t_repump*1e6, compound_geom_erlang_dist(t_repump, repump_success, singlet_lifetime))

In [ ]:
def p_singlet(t, r, tau, tau_exc):
    sum_cutoff = 1000
    pdf = np.zeros_like(t, dtype=np.double)
    geom_weight_sum = 0.0
    for i in xrange(1,sum_cutoff):
        # current_tau = tau + tau_exc if i > 1 else tau
        geom_weight = stats.geom.pmf(i, r)
        pdf += geom_weight * (1 - stats.erlang.cdf(t/tau, i))
        geom_weight_sum += geom_weight
    return pdf

In [ ]:
def p_repumped(t, r, tau, tau_exc):
    ps = p_singlet(t, r, tau, tau_exc)
    p0 = (1. - ps) + r*ps
    return p0

In [ ]:
def p_excited(t, tau_exc):
    return 1-stats.expon.pdf(t/tau_exc)

In [ ]:
def p_excited_repumped(t, r, tau, tau_exc):
    pr = p_repumped(t, r, tau, tau_exc)
    pexc = p_excited(t, tau_exc)
    
    return 1-(pexc*pr)

In [ ]:
plt.plot(t_repump*1e6, p_singlet(t_repump, repump_success, singlet_lifetime, excitation_time), label='singlet')
plt.plot(t_repump*1e6, p_repumped(t_repump, repump_success, singlet_lifetime, excitation_time), label='repump success')
plt.plot(t_repump*1e6, 1. - p_repumped(t_repump, repump_success, singlet_lifetime, excitation_time), label='repump failure')
plt.plot(t_repump*1e6, p_excited_repumped(t_repump, repump_success, singlet_lifetime, excitation_time), label='repump failure with excitation')
plt.plot(t_repump*1e6, p_excited(t_repump, excitation_time), label='excitation')

plt.legend()
plt.xlabel("repumping duration (us)")
plt.ylabel("probability")
plt.xlim(0,2)

In [ ]:
# lets try overlaying a repump time measurement with the above model
reload(pu_delayfb)

ot = "20170727114000"

repump_x, repump_y, repump_y_u = pu_delayfb.repump_speed(older_than=ot, ret_data=True)


In [ ]:
plt.errorbar(repump_x, repump_y, repump_y_u, fmt='o', label='repump failure')
# plt.plot(t_repump*1e9, p_excited(t_repump, excitation_time), label='excitation')
plt.plot(t_repump*1e9, p_excited_repumped(t_repump, repump_success, singlet_lifetime, excitation_time), label='repump failure with excitation', color='r')

plt.ylabel("repump failure rate")
plt.xlabel("repump duration")

plt.xlim(0,2000)

In [ ]:
exc_range = 30. # ns

#plt.errorbar(repump_x, np.log(repump_y), fmt='o')

exc_range_mask = repump_x < exc_range
repump_exc_x = repump_x[exc_range_mask]
repump_exc_y = repump_y[exc_range_mask]

plt.errorbar(repump_exc_x, np.log(repump_exc_y), fmt='o')

fr = fit.fit1d(repump_exc_x*1e-9, np.log(repump_exc_y), common.fit_line, 1, -1, do_print=True)
print(fr['fitfunc_str'])
gamma_exc = -fr['params_dict']['b']
print(gamma_exc / 1e6)
tau_exc = 1/gamma_exc
print(tau_exc*1e9)

# Gaussianized model

In [ ]:
a = 0.5
b = 9./11.
c = (1 - b)/b**2
d = a*(1-a)
tau = 300. # ns
L = 50
tau_exc = 0. # ns

n_tries = int(1e7)

In [ ]:
Z_R = stats.norm.rvs(size=n_tries)
Z_S = stats.norm.rvs(size=n_tries)
Z_Sp = stats.norm.rvs(size=n_tries)
Z_Tp = stats.norm.rvs(size=n_tries)
Z_Tm = stats.norm.rvs(size=n_tries)

In [ ]:
R = a*L + Z_R * np.sqrt(d*L)
S = R/b + Z_S * np.sqrt(c*R)
Sp = S/2 + Z_Sp * np.sqrt(S/4)
Tp = Sp*tau + Z_Tp * np.sqrt(Sp*tau**2)

Sm = S - Sp
Tm = Sm*tau + Z_Tm * np.sqrt(Sm*tau**2)

In [ ]:
valids = (R > 0.0) & (S > 0.0) & (Sp > 0.0) & (Tp > 0.0) & (Sm > 0.0) & (Tm > 0.0)
print("valid values: " + str(np.sum(valids)))

R = R[valids]
S = S[valids]
Sp = Sp[valids]
Tp = Tp[valids]
Sm = Sm[valids]
Tm = Tm[valids]

In [ ]:
plt.figure()
plt.hist(R, bins=100)
plt.figure()
plt.hist(S, bins=100)
plt.figure()
plt.hist(Sp, bins=100)
plt.figure()
plt.hist(Tp, bins=100)
plt.figure()
plt.hist(Sm, bins=100)
plt.figure()
plt.hist(Tm, bins=100)

plt.show()

In [ ]:
# anti-correlation check between Tp and Tm

Tp_dev = Tp - np.mean(Tp)
Tm_dev = Tm - np.mean(Tm)

corr = Tp_dev*Tm_dev

corr_cut = corr[(corr > -1e4) & (corr < 1e4)]

plt.figure()
plt.hist(corr_cut, bins=100)
plt.xlim(-1e3, 1e3)
plt.show()
plt.close('all')

In [ ]:
np.mean(corr)

In [ ]:
carbon = 2

freq_diffs = np.diff(freqs[carbon-1,:]) * np.array([-1.0, 1.0])

d_phi = (freq_diffs[0] * Tm*1e-9 + freq_diffs[1] * Tp*1e-9) * 360.0

In [ ]:
plt.hist(d_phi, bins=100)
plt.show()

print("mean phase: " + str(np.mean(d_phi)))
print("phase standard deviation: " + str(np.std(d_phi)))

In [ ]:
## import time

def calculate_phase_mean_stddev(a, b, c, d, tau, tau_exc, L, carbon):
    n_tries = int(1e6)
    
    Z_R = stats.norm.rvs(size=n_tries)
    Z_S = stats.norm.rvs(size=n_tries)
    Z_Sp = stats.norm.rvs(size=n_tries)
    Z_Tp = stats.norm.rvs(size=n_tries)
    Z_Tm = stats.norm.rvs(size=n_tries)
    Z_Te = stats.norm.rvs(size=n_tries)
    
    R = a*L + Z_R * np.sqrt(d*L)
    S = R/b + Z_S * np.sqrt(c*R)
    Sp = S/2 + Z_Sp * np.sqrt(S/4)
    Tp = Sp*tau + Z_Tp * np.sqrt(Sp) * tau

    Sm = S - Sp
    Tm = Sm*tau + Z_Tm * np.sqrt(Sm) * tau
    
    Te = 0.0 + Z_Te * np.sqrt(R) * tau_exc
    
    
    
    # valids = (R > 0.0) & (S > 0.0) & (Sp > 0.0) & (Tp > 0.0) & (Sm > 0.0) & (Tm > 0.0)
    validity_matrix = np.vstack((R, S, Sp, Tp, Sm, Tm))
    validity_matrix[np.isnan(validity_matrix)] = -1.
    valids = np.all(validity_matrix > 0.0, axis=0)
#     print("valid values: " + str(np.sum(valids)))
    
    R = R[valids]
    S = S[valids]
    Sp = Sp[valids]
    Tp = Tp[valids]
    Sm = Sm[valids]
    Tm = Tm[valids]
    Te = Te[valids]
    
    
    freq_diffs = np.diff(freqs[carbon,:]) * np.array([-1.0, 1.0])

    d_phi = (freq_diffs[0] * Tm*1e-9 + freq_diffs[1] * Tp*1e-9) * 360.0
    
    proj = np.cos(np.deg2rad(d_phi))
    
    
    
    return np.mean(d_phi), np.std(d_phi), np.mean(proj), np.std(proj)

In [ ]:
carbon_str = "2"
carbon = c_idxs[carbon_str]

a = 0.5
b = 9./11.
c = (1 - b)/b**2
d = a*(1-a)
tau = 300. # ns
L = 50
tau_exc = 0. # ns

# for carbon_str, carbon in c_idxs:
if True:
    print "Working on carbon " + carbon_str
    L = np.array([50,60,70,80,90,100,120,140,160,180,200,250,300,350,400,450,500,550,600])
    phi_means = np.zeros_like(L, dtype=np.double)
    phi_stds = np.zeros_like(L, dtype=np.double)

    proj_means = np.zeros_like(L, dtype=np.double)
    proj_std = np.zeros_like(L, dtype=np.double)

    for i_l, l in enumerate(L):
        print("Iteration %d/%d" % (i_l+1, len(L)))
        phi_means[i_l], phi_stds[i_l], proj_means[i_l], proj_std[i_l] = calculate_phase_mean_stddev(a, b, c, d, tau, tau_exc, l, carbon)

    plt.figure()
    plt.scatter(L, phi_means, label='mean')
    plt.scatter(L, phi_stds, label='std dev')
    plt.xlim(0,610)
    plt.ylim(0,120)
    plt.xlabel("number of LDE attempts")
    plt.ylabel("angle (degrees)")
    plt.title("repump phase statistics on C%s" % carbon_str)
    plt.legend()
    plt.savefig("plots/repump-phase-stats-T2star-C%s.png" % carbon_str, dpi=300)

    plt.figure()
    plt.scatter(L, np.deg2rad(phi_stds)**2)
    fr = fit.fit1d(L, np.deg2rad(phi_stds)**2, common.fit_line, 0, 0.01, do_print=True, fixed=[0])
    decay_N = 2. / fr['params_dict']['b']
    plt.plot(np.linspace(0,600), fr['fitfunc'](np.linspace(0,600)), label='N = %.0f' % decay_N)
    plt.savefig("plots/repump-sigma-T2star-C%s.png" % carbon_str, dpi=300)

    decay_fun = lambda mu, sigma: np.cos(mu)*np.exp(-0.5*sigma**2)

    plt.figure()

    # plt.figure(figsize=(10,10))
    # plt.scatter(L, proj_means, label='sim. mean')
    # plt.scatter(L, proj_std, label='sim. std dev')
    # plt.scatter(L, decay_fun(np.deg2rad(phi_means), np.deg2rad(phi_stds)), label='mu + sigma decay')
    plt.scatter(L, decay_fun(np.deg2rad(0.0), np.deg2rad(phi_stds)), label=r'sigma decay, N = %.f' % decay_N)
    # plt.scatter(L, decay_fun(np.deg2rad(phi_means), np.deg2rad(0.0)), label='mu decay')

    if carbon < len(carbons):
        c_data = single_C_decay_data[int(carbon_str)]


    #     T2star_corrfactors = 1. / T2star_envelope(c_data["x"]*LDE_duration, 20e-3)
    #     plt.errorbar(c_data["x"], c_data["y"]*T2star_corrfactors, c_data["y_u"]*T2star_corrfactors, fmt='o', label="meas. data, T2* corrected", color="C5")
        LDE_duration = 7e-6
        plt.errorbar(
            c_data["x"], c_data["y"], c_data["y_u"], fmt='o', 
            label='T2* corr. meas. data, N = %.f' % c_data['fr']['params_dict']['T'],
            color='r'
        )

    plt.legend()
    plt.xlim(0,610)
    plt.ylim(0,1.1)
    plt.xlabel("number of LDE attempts")
    plt.ylabel(r"$\langle X \rangle$")
    plt.title("nuclear dephasing for C%s, index %s" % (carbon_str, carbon))
    plt.savefig("plots/repump-decay-T2star-C%s.png" % carbon_str, dpi=300)

In [ ]:
taus = stats.expon.rvs(scale=tau, size=n_tries) * 1e-9
omega_choices = stats.bernoulli.rvs(0.5,size=n_tries) * 2. - 1.

cfreqs = freqs[carbon,:]
avg_pm_freq = np.mean(cfreqs[[0,2]])
diff_pm_freq = np.diff(cfreqs[[0,2]])[0]
diff_pm_omega = diff_pm_freq * 2 * np.pi

omegas = (0.5 * diff_pm_freq * omega_choices) * 2 * np.pi

single_phis = omegas * taus

plt.hist(single_phis, bins=100)
plt.show()

In [ ]:
single_phi_var = np.var(single_phis)
print single_phi_var

In [ ]:
calc_single_phi_var = (diff_pm_omega ** 2) / 2 * (tau * 1e-9)**2
print(calc_single_phi_var)

In [ ]:
2. / calc_single_phi_var

In [ ]:
carbon

In [ ]:
%timeit -n 10 Z_R = stats.norm.rvs(size=int(1e6))

In [ ]:
for ccs in ['13+', '1', '3']:
    print (freqs[c_idxs[ccs],:])

In [ ]:
a

In [ ]:
def naive_fidelity(N, sigma_phi):
    return 0.5+0.5*np.exp(-.5*(N/2)*(sigma_phi)**2)

def faraday_fidelity(N, sigma_phi):
    return 0.5+((1/2.)**(N+1))*(1+np.exp(-0.5*(sigma_phi)**2))**N

In [ ]:
sigma_phi = 0.05

Nvals = np.arange(1,600)

plt.plot(Nvals, naive_fidelity(Nvals, sigma_phi), label='naive')
plt.plot(Nvals, faraday_fidelity(Nvals, sigma_phi), label='faraday')

plt.legend()

plt.figure()
plt.plot(Nvals, naive_fidelity(Nvals, sigma_phi)-faraday_fidelity(Nvals, sigma_phi))

